## Load the image data
The image is enhanced in contrast and brightness to remove background noise. 
The resulting image should show the digit on a white background:

In [76]:
import cv2
import numpy as np
from PIL import Image as PIL_Image
from PIL import ImageEnhance
from PIL import ImageOps
from PIL import ImageDraw
import math
from scipy import misc
from array import *
import imageio

#original captured image
img_path = 'class/'
img_name = 'digit-9c.jpg'
img = PIL_Image.open(img_path + img_name).convert("L")   
                   
# Center crop to remove noisy edges
width, height = img.size   # Get dimensions
new_width = width * 0.8
new_height = height * 0.8

left = (width - new_width)/2
top = (height - new_height)/2
right = (width + new_width)/2
bottom = (height + new_height)/2

img = img.crop((left, top, right, bottom))

# Add border to allow for dilation
img = ImageOps.expand(img,border=150,fill='white') 

# Dilate image. Must first convert to openCV
cv_image = np.asarray(img) 
# Convert RGB to BGR 
cv_image = cv_image[ ::-1].copy() 
kernel = np.ones((50,50), np.uint8)
#print(cv_image.size)
img_dilation = cv2.erode(cv_image, kernel, iterations=1)
#img_dilation = cv2.dilate(cv_image, Mat(), Point(-1, -1), 2, 1, 1)
#cv2.imshow('img', img_dilation)
#cv2.waitKey(0)
img = PIL_Image.fromarray(img_dilation)
#print(img.size)

#Image enhancement                
contr = ImageEnhance.Contrast(img)
img = contr.enhance(3)                      # The enhancement values (contrast and brightness) 
bright = ImageEnhance.Brightness(img)       # depends on backgroud, external lights etc
img = bright.enhance(4.0)          

im2 = img.convert('RGBA')
# rotated image
rot = im2.rotate(-90, expand=1)
# a white image same size as rotated image
fff = PIL_Image.new('RGBA', rot.size, (255,)*4)
# create a composite image using the alpha layer of rot as a mask
out = PIL_Image.composite(rot, fff, rot)

img = out.convert('RGB')

#Find bounding box  
inverted = ImageOps.invert(img)  
box = inverted.getbbox()  
#print(box)
#draw = ImageDraw.Draw(img)
#draw.rectangle(box, fill=None, outline="green")
#img.show()

img_new = img.crop(box)  
width, height = img_new.size  
ratio = min((28./height), (28./width))  
background = PIL_Image.new('RGB', (28,28), (255,255,255))  
if(height == width):  
    img_new = img_new.resize((28,28))  
elif(height>width):  
    img_new = img_new.resize((int(width*ratio),28))  
    background.paste(img_new, (int((28-img_new.size[0])/2),int((28-img_new.size[1])/2)))  
else:  
    img_new = img_new.resize((28, int(height*ratio)))  
    background.paste(img_new, (int((28-img_new.size[0])/2),int((28-img_new.size[1])/2)))  

# Create small image    
img_load = background
print(img_load.size)    
    
#Resize the image and invert it (white on black)  
smallimg = ImageOps.invert(img_load)  
smallimg = smallimg.rotate(0)  
smallimg = smallimg.transpose(PIL_Image.FLIP_LEFT_RIGHT)


out_path = 'out/'
imageio.imwrite(out_path + img_name, smallimg) 
smallimg

(28, 28)
